In [1]:
import pandas as pd
df = pd.read_csv("../../generated/dataframes/Jakku_1.csv")

In [2]:
df

,game,event,timestamp,confidence,video
0,bf2,kill_brc,00:01:51:04,0.979,Jakku
1,bf2,kill_brc,00:01:53:29,0.979,Jakku
2,bf2,kill_brc,00:02:03:26,0.980,Jakku
3,bf2,kill_brc,00:02:18:28,0.980,Jakku
4,bf2,kill_brc,00:02:31:24,0.977,Jakku
5,bf2,kill_brc,00:02:33:28,0.977,Jakku
6,bf2,kill_brc,00:02:38:05,0.981,Jakku
7,bf2,kill_brc,00:03:02:29,0.978,Jakku
8,bf2,kill_brc,00:03:17:21,0.980,Jakku
9,bf2,kill_brc,00:04:02:03,0.985,Jakku


In [3]:
df["event"].value_counts()

event
kill_brc    49
Name: count, dtype: int64

In [18]:
df.groupby('event')['confidence'].mean()


event
ag             1.000000
death          0.921000
game_menu      1.000000
heavy          0.991600
hero           0.991000
kill_brc       0.996132
kill_sgt       0.996402
loadout        0.894545
main_menu      1.000000
match_end      0.995000
podium         0.999000
scoreboard     0.998833
sniper         0.997667
spawn_troop    0.993000
Name: confidence, dtype: float64

In [19]:
df.groupby('event')['confidence'].agg(['min', 'max'])

,min,max
event,,
ag,1.000,1.000
death,0.900,0.941
game_menu,1.000,1.000
heavy,0.990,0.994
hero,0.991,0.991
kill_brc,0.993,0.997
kill_sgt,0.990,0.999
loadout,0.819,0.975
main_menu,1.000,1.000


In [20]:
# Convert timestamp to timedelta (handle "HH:MM:SS:FF" as if "HH:MM:SS:frames")
def timestamp_to_seconds(ts):
    h, m, s, f = map(int, ts.split(":"))
    return h * 3600 + m * 60 + s + f / 30  # Assuming 30 fps

df['timestamp_sec'] = df['timestamp'].apply(timestamp_to_seconds)

# Extract timestamps
spawn_times = df[df['event'] == 'spawn_troop']['timestamp_sec'].values
loadout_times = df[df['event'] == 'loadout']['timestamp_sec'].values

# Compute closest preceding loadout for each spawn
delays = []
for spawn in spawn_times:
    prior_loadouts = loadout_times[loadout_times < spawn]
    if len(prior_loadouts) > 0:
        delay = spawn - prior_loadouts[-1]  # last loadout before spawn
        delays.append(delay)

avg_delay = sum(delays) / len(delays) if delays else None
print(f"Average delay between loadout and spawn: {avg_delay:.2f} seconds")


Average delay between loadout and spawn: 44.15 seconds


In [28]:
# Get only relevant rows and sort chronologically
life_df = df[df['event'].isin(['spawn_troop', 'loadout', 'kill_brc', 'kill_sgt', 'kill'])]
life_df = life_df.sort_values(by='timestamp')

# Reset index to allow iteration
life_df = life_df.reset_index(drop=True)

# Track kills between each spawn and the next loadout
kd_ratios = []
current_kill_count = 0
in_life = False

for _, row in life_df.iterrows():
    event = row['event']
    
    if event == 'spawn_troop':
        current_kill_count = 0
        in_life = True

    elif event == 'kill_brc' and in_life:
        current_kill_count += 1

    elif event == 'loadout' and in_life:
        kd_ratios.append(current_kill_count)
        in_life = False

# Compute average K/D
avg_kd = sum(kd_ratios) / len(kd_ratios) if kd_ratios else 0
print(f"Average K/D ratio: {avg_kd:.2f}")


Average K/D ratio: 3.80


In [29]:
# Filter and sort relevant events
life_df = df[df['event'].isin(['spawn_troop', 'loadout', 'kill_brc'])]
life_df = life_df.sort_values(by='timestamp').reset_index(drop=True)

# Group kills per life, record spawn timestamps
life_kills = []
spawn_timestamps = []
current_kills = []
in_life = False
current_spawn_time = None

for _, row in life_df.iterrows():
    event = row['event']
    
    if event == 'spawn_troop':
        current_kills = []
        current_spawn_time = row['timestamp']
        in_life = True
    
    elif event.startswith('kill') and in_life:
        current_kills.append(event)
    
    elif event == 'loadout' and in_life:
        life_kills.append(current_kills)
        spawn_timestamps.append(current_spawn_time)
        in_life = False

# Print kills and spawn timestamps
for i, (kills, spawn_time) in enumerate(zip(life_kills, spawn_timestamps), 1):
    print(f"🧬 Life {i:<2} | Spawned at {spawn_time} | {len(kills)} kills")


🧬 Life 1  | Spawned at 00:05:03:05 | 0 kills
🧬 Life 2  | Spawned at 00:05:18:00 | 5 kills
🧬 Life 3  | Spawned at 00:07:58:05 | 9 kills
🧬 Life 4  | Spawned at 00:11:49:24 | 16 kills
🧬 Life 5  | Spawned at 00:19:03:02 | 0 kills
🧬 Life 6  | Spawned at 00:19:38:09 | 1 kills
🧬 Life 7  | Spawned at 00:20:37:13 | 1 kills
🧬 Life 8  | Spawned at 00:20:55:22 | 2 kills
🧬 Life 9  | Spawned at 00:21:45:22 | 3 kills
🧬 Life 10 | Spawned at 00:24:27:24 | 1 kills


In [24]:
df

,game,event,timestamp,confidence,video,timestamp_sec
0,bf2,main_menu,00:00:02:00,1.000,Jakku,2.000000
1,bf2,ag,00:00:36:20,1.000,Jakku,36.666667
2,bf2,loadout,00:00:50:00,0.917,Jakku,50.000000
3,bf2,game_menu,00:01:20:15,1.000,Jakku,80.500000
4,bf2,spawn_troop,00:01:32:13,0.992,Jakku,92.433333
...,...,...,...,...,...,...
190,bf2,kill_sgt,00:24:50:20,0.998,Jakku,1490.666667
191,bf2,scoreboard,00:24:55:28,1.000,Jakku,1495.933333
192,bf2,podium,00:25:16:00,0.999,Jakku,1516.000000
193,bf2,match_end,00:25:20:00,0.995,Jakku,1520.000000
